In [2]:
import webif as quickdb

### Count all objects

In [ ]:
sql = '''
SELECT
    count(*)
FROM
    pdr2_wide
'''

%time quickdb.dataframe(sql)

In [ ]:
sql = '''
SELECT
    count(*)
FROM
    pdr2_wide
GROUP BY
    forced.universal.isprimary
'''

%time quickdb.dataframe(sql)

### Magnitude Histogram

In [ ]:
sql = '''
    SELECT
        histogram(forced.i.psfflux_mag, bins => 200, range => (10, 30)) as hist
    FROM
        pdr2_wide
    WHERE
        forced.universal.isprimary
    GROUP BY
        forced.i.extendedness_value < 0.5
'''

%time hist1 = quickdb.dataframe(sql)
hist1

%matplotlib inline
from matplotlib import pyplot

for i, row in hist1.iterrows():
    hist, bins = row['hist']
    group = row['group']
    pyplot.plot(bins[1:], hist, label=f'group: {group}')
pyplot.grid()
pyplot.legend()

## 2D Histogram

### Stellar sequence

In [ ]:
sql = f'''
SELECT
    histogram2d(
        forced.g.psfflux_mag - forced.r.psfflux_mag,
        forced.r.psfflux_mag - forced.i.psfflux_mag,
        bins => (200, 400),
        range => ((0, 1.5), (-0.5, 2.5))
    )
FROM
    pdr2_dud
WHERE
    forced.universal.isprimary
    AND
    forced.i.extendedness_value < 0.5
'''

%time stellarsequence = quickdb.dataframe(sql)
display(stellarsequence)

import numpy

hist, xedges, yedges = stellarsequence['c0'][0]
pyplot.imshow(numpy.log(1 + hist.T), origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])